In [2]:
import numpy as np #operaciones matriciales y con vectores
import pandas as pd #tratamiento de datos
import matplotlib.pyplot as plt #gráficos
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LinearRegression # modelos lineales
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split #metodo de particionamiento de datasets para evaluación
from sklearn.model_selection import cross_val_score #protocolo de evaluación
from sklearn import datasets, metrics
from sklearn import preprocessing
from sklearn.metrics import make_scorer
import warnings
warnings.filterwarnings('ignore')

In [14]:
col_Headers = ["EventType","RegisterDate","StopID","Odometer","Longitude","Latitude", "TaskID","LineID","TripID","DatagramID","DatagramDate","DatagramTime","BusID"]
df = pd.read_csv('DATAGRAMS.csv',',',dtype={"EventType": str, "RegisterDate": str, "StopID": str,"Odometer": str,"Longitude": str,"Latitude": str, "TaskID": str,"LineID": str,"TripID": str,"DatagramID": str,"DatagramDate": str,"DatagramTime": str,"BusID": str},names = col_Headers)

## Analisis Exploratorio de Datos

In [7]:
df.head()

,EventType,RegisterDate,StopID,Odometer,Longitude,Latitude,TaskID,LineID,TripID,DatagramID,DatagramDate,Datagram Time,BusID
0,19,2018-09-04,501051,1,-764871900,34451717,572,140,548,5661359158,2018-09-03,07:05:57,714
1,19,2018-09-04,502103,1063,-765463867,33977717,1023,217,882,5661362653,2018-09-03,07:07:19,311
2,7,2018-09-04,515152,24,-765076783,34191300,705,2471,43,5661362655,2018-09-03,07:07:19,796
3,19,2018-09-04,502650,788,-764913067,34220317,143,150,389,5661367800,2018-09-03,07:09:21,561
4,7,2018-09-04,515006,91,-765005167,34115933,705,2471,43,5661348896,2018-09-03,07:01:54,796


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2242473 entries, 0 to 2242472
Data columns (total 13 columns):
EventType       object
RegisterDate    object
StopID          object
Odometer        object
Longitude       object
Latitude        object
TaskID          object
LineID          object
TripID          object
DatagramID      object
DatagramDate    object
DatagramTime    object
BusID           object
dtypes: object(13)
memory usage: 222.4+ MB


In [8]:
df.shape

(2242473, 13)

In [16]:
df.describe()

,EventType,RegisterDate,StopID,Odometer,Longitude,Latitude,TaskID,LineID,TripID,DatagramID,DatagramDate,DatagramTime,BusID
count,2242473,1528688,2242473,2242473,2242473,2242473,2242473,2242471,2242471,2242471,1523099,2242471,1523099
unique,33,245,2122,12573,65755,108932,4306,111,2798,2242471,242,76801,853
top,7,2019-03-30,-1,-1,-1,-1,-1,2273,-1,5801319415,2019-03-14,147,147
freq,320439,11502,78210,524456,76703,76703,62608,101770,64520,1,12400,17579,35214


In [17]:
df.describe(include = np.object)

,EventType,RegisterDate,StopID,Odometer,Longitude,Latitude,TaskID,LineID,TripID,DatagramID,DatagramDate,DatagramTime,BusID
count,2242473,1528688,2242473,2242473,2242473,2242473,2242473,2242471,2242471,2242471,1523099,2242471,1523099
unique,33,245,2122,12573,65755,108932,4306,111,2798,2242471,242,76801,853
top,7,2019-03-30,-1,-1,-1,-1,-1,2273,-1,5801319415,2019-03-14,147,147
freq,320439,11502,78210,524456,76703,76703,62608,101770,64520,1,12400,17579,35214


### Se encontro los siguientes eventos en los datagramas

In [10]:
df['EventType'].value_counts()

0      9306959
7         3667
17        2995
19        2855
9         2536
26        1974
8         1911
11        1845
23        1620
22        1606
2          878
5          837
10         773
18         632
103        575
25         547
24         537
3          413
13         210
21         156
1          152
6          144
12         119
29          62
20          56
27          37
28          36
16          34
14          21
4           20
15          17
31          12
30           2
Name: EventType, dtype: int64

#### Importancia del aire sobre todo el sistema

Se hicieron las siguientes observaciones:
 1. Se encontraron que de 9334238 registros de EventTypes, de los cuales 9306959 (99,7% del total de datos disponibles) pertencen al EventType 0, el cual representa que una notificacion de ingreso al datagrama que no tiene ningun evento relevante.
 1. Decidimos enfocarnos en los datagramas que no tuvieran EventType 0, entonces encontramos que en total son 27279 datos.
 1. Se decidio acotar unicamente a los registros que equivalen a fallas del vehiculo, cuyos EventTypes son: 13, 12, 11, 14, 21 y 23. Lo anterior corresponde a un total de 3971 registros.
 1. Los eventos 12 y 23, corresponden a fallas del aire acondicionado. El evento 12 equivale a 'Presencia de falla específica del aire acondicionado del vehículo' y el 23 a 'Mensaje automático de MEMO.' correspondiente a una falla de aire acondicionado.
 1. Existen 119 registros del evento 12 y 1620 del 23, es decir que el total de estos registros representarian el 6.37% del total de datos de EventTypes diferentes de 0.
 1. Volviendo a calcular el porcentaje de registros correspondientes a fallas de aire acondicionado vs el total en fallas en vehiculo, se tiene que la fallas de aire acondicionado son el 43% de fallas en el vehiculo.
 1. El EventType 11 corresponde al evento 'Falla en vehiculo', cuya descripcion es la siguiente 'Presencia de una falla que permite que el vehículo se movilice hasta algún punto donde pueda ser atendido o ser retirado de la operación.', este evento tiene mayor cantidad de registros que las fallas de aire acondicionado, sin embargo no exta claro si una falla de aire acondicionado conlleva a una falla en vehiculo o viceversa.

Procederemos a hacer seguimiento de los buses y de las fechas

### Seguimiento de fechas de registro

In [11]:
df['RegisterDate'].value_counts()

02-JUN-18    2329672
02-NOV-18    2320695
03-NOV-18    2083665
03-JUN-18    1742833
01-NOV-18     487313
01-JUN-18     370060
Name: RegisterDate, dtype: int64

In [12]:
df[(df.EventType == '12') | (df.EventType == '23')]


,EventType,RegisterDate,StopID,Odometer,Latitude,Longitude,TaskID,LineID,TripID,DatagramID,DatagramDate,BusID
1147,23,01-NOV-18,500251,-1,34759050,-765043300,1040,2301,14,5792073657,01-NOV-18 05.37.28.000000 AM,638
3731,23,01-NOV-18,511222,153,33720250,-765246650,288,2121,24,5792071177,01-NOV-18 05.36.25.000000 AM,151
6984,23,02-NOV-18,504008,-1,34884750,-764935183,557,2121,2,5794674771,02-NOV-18 04.50.25.000000 AM,1174
13550,23,02-NOV-18,500158,-1,34437433,-764831267,229,2242,275,5794668031,02-NOV-18 04.45.43.000000 AM,153
19161,23,02-NOV-18,500151,24,34439967,-764837550,480,2402,3,5794662599,02-NOV-18 04.41.37.000000 AM,1095
...,...,...,...,...,...,...,...,...,...,...,...,...
9289297,23,03-JUN-18,513325,87,34829250,-765014933,268,2121,2097,5452511712,02-JUN-18 05.30.49.000000 PM,809
9312917,12,03-JUN-18,512044,268,34844400,-765147967,587,2274,508,5452458873,02-JUN-18 05.00.13.000000 PM,640
9322300,23,03-JUN-18,-1,-1,33791133,-765244767,268,-1,1667,5453206668,02-JUN-18 09.22.16.000000 AM,809
9325114,23,03-JUN-18,512178,54,34622050,-765292567,1,305,304,5453196609,02-JUN-18 09.17.00.000000 AM,638


In [13]:
df.BusID.unique().size

832

In [14]:
df[(df.EventType == '12') | (df.EventType == '23')].BusID.unique().size

103

Tenemos 103 buses que han presentado fallas en el aire acondicionado, lo que equivale al 12% de la flota. Además, tenemos 1739 registros de fallas de aire acondicionado, suponiendo que se hayan presentado equitativamente implicaria que cada bus, ha fallado 16 veces.

### Analisis BusID 640

In [15]:
df[(((df.EventType == '12') | (df.EventType == '23')) & (df.BusID == '640'))& (df.RegisterDate == '01-NOV-18') ]

,EventType,RegisterDate,StopID,Odometer,Latitude,Longitude,TaskID,LineID,TripID,DatagramID,DatagramDate,BusID
85245,23,01-NOV-18,500107,664,34825100,-765043500,638,321,19,5792133747,01-NOV-18 06.02.38.000000 AM,640
239808,23,01-NOV-18,500107,116,34833283,-765092483,638,321,13,5792076308,01-NOV-18 05.38.37.000000 AM,640
281602,23,01-NOV-18,513041,-1,34932283,-764924067,638,321,339,5792749810,01-NOV-18 04.40.38.000000 AM,640
1466207,23,01-NOV-18,513044,5927,34920000,-764924483,638,321,339,5792782956,01-NOV-18 05.01.39.000000 AM,640
1784205,23,01-NOV-18,513041,-1,34928383,-764921817,638,321,350,5792827595,01-NOV-18 05.23.37.000000 AM,640
4062642,23,01-NOV-18,513044,3279,34827533,-765053350,638,321,339,5792768964,01-NOV-18 04.53.38.000000 AM,640


In [16]:
df[(((df.EventType == '12') | (df.EventType == '23')) & (df.BusID == '640'))& (df.RegisterDate == '02-NOV-18') ]

,EventType,RegisterDate,StopID,Odometer,Latitude,Longitude,TaskID,LineID,TripID,DatagramID,DatagramDate,BusID
708699,23,02-NOV-18,514414,113,34575367,-764833733,150,2243,311,5794798924,02-NOV-18 05.49.19.000000 AM,640
748102,23,02-NOV-18,513013,1190,34796933,-764945217,150,2243,311,5794818175,02-NOV-18 05.57.19.000000 AM,640
1122164,23,02-NOV-18,513256,873,34830567,-765062933,477,2243,353,5795335027,01-NOV-18 08.54.47.000000 PM,640
1206521,23,02-NOV-18,514465,316,34162450,-764636917,477,2243,353,5795253588,01-NOV-18 08.16.47.000000 PM,640
1300364,23,02-NOV-18,514010,99,34567067,-764838967,150,2243,361,5794896682,02-NOV-18 06.28.20.000000 AM,640
1972904,23,02-NOV-18,514465,-1,34134333,-764675367,150,2243,311,5794725148,02-NOV-18 05.17.19.000000 AM,640
2771498,23,02-NOV-18,500103,-1,34828217,-765085867,150,2243,361,5794842926,02-NOV-18 06.07.20.000000 AM,640
3652688,23,02-NOV-18,514401,35,34456183,-764839983,477,2243,353,5795295221,01-NOV-18 08.35.47.000000 PM,640
3670739,23,02-NOV-18,513103,1817,34752417,-764890367,477,2243,353,5795314119,01-NOV-18 08.44.46.000000 PM,640


In [17]:
df[(((df.EventType == '12') | (df.EventType == '23')) & (df.BusID == '640'))& (df.RegisterDate == '03-NOV-18') ]

,EventType,RegisterDate,StopID,Odometer,Latitude,Longitude,TaskID,LineID,TripID,DatagramID,DatagramDate,BusID
3828006,23,03-NOV-18,514156,280,34380183,-764842550,150,2243,370,5798790711,02-NOV-18 08.15.18.000000 AM,640
3982485,23,03-NOV-18,513325,325,34761183,-765005950,150,2243,370,5799263092,02-NOV-18 07.56.19.000000 AM,640
4044441,23,03-NOV-18,514013,3700,34821983,-765028233,150,2243,329,5798942027,02-NOV-18 09.16.20.000000 AM,640
4144223,23,03-NOV-18,515087,34,33967850,-765060750,813,2524,653,5797631591,02-NOV-18 07.58.14.000000 PM,640
4215687,23,03-NOV-18,515139,73,33933350,-765131050,813,2524,653,5797650317,02-NOV-18 08.06.14.000000 PM,640
4324510,23,03-NOV-18,514465,-1,-1,-1,150,2243,329,5798838809,02-NOV-18 08.34.24.000000 AM,640
4512533,23,03-NOV-18,514465,73,34116083,-764660667,150,2243,320,5799115582,02-NOV-18 06.58.19.000000 AM,640
4555128,23,03-NOV-18,514032,274,34121483,-764677200,150,2243,361,5799107933,02-NOV-18 06.55.20.000000 AM,640
4668139,23,03-NOV-18,514467,241,34313683,-764735283,150,2243,361,5799080052,02-NOV-18 06.44.20.000000 AM,640
4695006,23,03-NOV-18,501551,-1,34470683,-765294750,813,2524,246,5798240846,02-NOV-18 04.58.14.000000 PM,640


### Analisis BusID 153

In [23]:
df[(df.EventType == '12') & (df.BusID == '1069')]

,EventType,RegisterDate,StopID,Odometer,Latitude,Longitude,TaskID,LineID,TripID,DatagramID,DatagramDate,BusID
81238,12,01-NOV-18,514011,266,34628767,-764832067,588,2212,399,5792131141,01-NOV-18 06.01.35.000000 AM,1069
382654,12,02-NOV-18,513127,564,34648100,-765010383,301,2402,439,5795760624,01-NOV-18 04.39.22.000000 PM,1069
3935847,12,02-NOV-18,514156,41,34467783,-764840850,301,2402,439,5795805443,01-NOV-18 04.58.17.000000 PM,1069


In [20]:
df[(((df.EventType == '12') | (df.EventType == '23')) & (df.BusID == '153'))& (df.RegisterDate == '02-NOV-18')].sort_values(by = 'DatagramDate')

,EventType,RegisterDate,StopID,Odometer,Latitude,Longitude,TaskID,LineID,TripID,DatagramID,DatagramDate,BusID
1825530,23,02-NOV-18,513254,-1,34891667,-765051967,290,2274,200,5796130523,01-NOV-18 03.35.14.000000 PM,153
394382,23,02-NOV-18,513254,-1,34892800,-765056717,290,2274,245,5795455164,01-NOV-18 06.06.14.000000 PM,153
2352536,23,02-NOV-18,517117,-1,34121817,-765474300,290,2274,245,5795612207,01-NOV-18 07.13.15.000000 PM,153
2545463,23,02-NOV-18,519031,-1,34412767,-765329517,290,2274,368,5796751486,01-NOV-18 07.41.14.000000 AM,153
1266617,23,02-NOV-18,517208,-1,34236283,-765412000,290,2274,64,5796486821,01-NOV-18 09.22.14.000000 AM,153
4623928,23,02-NOV-18,501900,-1,34063350,-765472033,290,2274,64,5796517187,01-NOV-18 09.36.14.000000 AM,153
13550,23,02-NOV-18,500158,-1,34437433,-764831267,229,2242,275,5794668031,02-NOV-18 04.45.43.000000 AM,153
171527,23,02-NOV-18,514133,-1,34524983,-764893367,229,2242,275,5794683267,02-NOV-18 04.55.42.000000 AM,153
288152,23,02-NOV-18,513224,-1,34683333,-765090400,229,2242,275,5794699322,02-NOV-18 05.04.42.000000 AM,153
924801,23,02-NOV-18,512015,-1,34800183,-765203067,229,2242,275,5794713435,02-NOV-18 05.11.43.000000 AM,153


In [71]:
df[(((df.EventType == '12') | (df.EventType == '23')) & (df.BusID == '153'))& (df.RegisterDate == '03-NOV-18') ]

,EventType,RegisterDate,StopID,Odometer,Latitude,Longitude,TaskID,LineID,TripID,DatagramID,DatagramDate,BusID
2663011,23,03-NOV-18,500103,-1,-1,-1,20288,2121,99993,5797835108,02-NOV-18 05.45.08.000000 PM,153
2795225,23,03-NOV-18,500106,-1,34892967,-765077067,996,421,1005,5797289366,02-NOV-18 09.25.01.000000 PM,153
2818551,23,03-NOV-18,502400,-1,34635567,-765253717,996,421,1005,5797308556,02-NOV-18 09.36.01.000000 PM,153
2838532,23,03-NOV-18,500158,-1,34433517,-764824100,229,2242,327,5799241371,02-NOV-18 07.47.42.000000 AM,153
2841331,23,03-NOV-18,514156,-1,34432200,-764840733,229,2242,39,5799231143,02-NOV-18 07.43.42.000000 AM,153
3046944,23,03-NOV-18,514465,-1,-1,-1,615,2243,339,5797776409,02-NOV-18 05.20.09.000000 PM,153
3213651,23,03-NOV-18,512130,-1,34808233,-765228733,229,2242,39,5799157714,02-NOV-18 07.14.43.000000 AM,153
3218503,23,03-NOV-18,513328,-1,34709383,-765108700,229,2242,39,5799185629,02-NOV-18 07.25.43.000000 AM,153
3570571,23,03-NOV-18,502501,-1,34793250,-765165567,996,421,472,5797269468,02-NOV-18 09.14.01.000000 PM,153
3585336,23,03-NOV-18,501502,-1,34527150,-765300333,996,421,472,5797229247,02-NOV-18 08.53.01.000000 PM,153


### Analisis BusID 397

In [79]:
df[(df.RegisterDate=='02-NOV-18')& (df.BusID == '397')]

,EventType,RegisterDate,StopID,Odometer,Latitude,Longitude,TaskID,LineID,TripID,DatagramID,DatagramDate,BusID
6505,0,02-NOV-18,503006,2335,33674000,-765249000,809,1571,9940,5794674613,02-NOV-18 04.50.19.000000 AM,397
7049,0,02-NOV-18,503006,2335,33674133,-765249183,809,1571,9940,5794669686,02-NOV-18 04.46.55.000000 AM,397
7651,0,02-NOV-18,503006,2335,33673983,-765248983,809,1571,9940,5794673183,02-NOV-18 04.49.21.000000 AM,397
8277,0,02-NOV-18,503006,2335,33673983,-765249017,809,1571,9940,5794675315,02-NOV-18 04.50.47.000000 AM,397
8486,0,02-NOV-18,503006,2335,33673967,-765248967,809,1571,9940,5794671767,02-NOV-18 04.48.23.000000 AM,397
...,...,...,...,...,...,...,...,...,...,...,...,...
4661222,0,02-NOV-18,501801,33,34189667,-765483217,505,1571,746,5796538685,01-NOV-18 09.46.42.000000 AM,397
4662575,0,02-NOV-18,501850,-1,34156683,-765491983,505,1571,746,5796531888,01-NOV-18 09.43.23.000000 AM,397
4662791,0,02-NOV-18,501850,118,34141917,-765485133,505,1571,746,5796535431,01-NOV-18 09.45.06.000000 AM,397
4662843,0,02-NOV-18,501850,18,34156033,-765482467,505,1571,746,5796537090,01-NOV-18 09.45.55.000000 AM,397


In [80]:
df[(df.RegisterDate=='03-NOV-18')& (df.BusID == '397')]

,EventType,RegisterDate,StopID,Odometer,Latitude,Longitude,TaskID,LineID,TripID,DatagramID,DatagramDate,BusID
974763,0,03-NOV-18,501850,6348,33483117,-765240467,872,1571,9948,5797386858,02-NOV-18 10.26.47.000000 PM,397
975171,0,03-NOV-18,501850,-1,34077867,-765473617,872,1571,9948,5797366304,02-NOV-18 10.12.20.000000 PM,397
975455,0,03-NOV-18,501850,-1,33952517,-765462717,872,1571,9948,5797369796,02-NOV-18 10.14.45.000000 PM,397
975596,0,03-NOV-18,501850,1198,33839400,-765445000,872,1571,9948,5797373146,02-NOV-18 10.17.05.000000 PM,397
978920,0,03-NOV-18,501850,7183,33484917,-765239067,872,1571,9948,5797389510,02-NOV-18 10.28.44.000000 PM,397
...,...,...,...,...,...,...,...,...,...,...,...,...
4750287,0,03-NOV-18,501751,-1,34232817,-765470750,872,1571,1093,5797227766,02-NOV-18 08.52.17.000000 PM,397
4750412,0,03-NOV-18,501801,-1,34187367,-765483217,872,1571,1093,5797224362,02-NOV-18 08.50.34.000000 PM,397
4750449,0,03-NOV-18,501801,116,34197850,-765482750,872,1571,1093,5797226003,02-NOV-18 08.51.24.000000 PM,397
4766809,0,03-NOV-18,501801,32,34190100,-765483183,872,1571,1093,5797225029,02-NOV-18 08.50.54.000000 PM,397


In [24]:
df[(df.EventType=='11')& (df.BusID == '397')]

,EventType,RegisterDate,StopID,Odometer,Latitude,Longitude,TaskID,LineID,TripID,DatagramID,DatagramDate,BusID
